In [1]:
import requests
import pandas as pd
from datetime import datetime

In [2]:
API_URL = "http://localhost:8000"
API_V1 = f"{API_URL}/api/v1"

# эндпоинты
HEALTH_ENDPOINT = f"{API_V1}/health"
FORWARD_ENDPOINT = f"{API_V1}/forward"
HISTORY_ENDPOINT = f"{API_V1}/history"
STATS_ENDPOINT = f"{API_V1}/stats"
LOGIN_ENDPOINT = f"{API_V1}/auth/login"
ME_ENDPOINT = f"{API_V1}/auth/me"

print(f"Health: {HEALTH_ENDPOINT}")
print(f"Forward: {FORWARD_ENDPOINT}")
print(f"History: {HISTORY_ENDPOINT}")
print(f"Stats: {STATS_ENDPOINT}")
print(f"Login: {LOGIN_ENDPOINT}")
print(f"Me: {ME_ENDPOINT}")

Health: http://localhost:8000/api/v1/health
Forward: http://localhost:8000/api/v1/forward
History: http://localhost:8000/api/v1/history
Stats: http://localhost:8000/api/v1/stats
Login: http://localhost:8000/api/v1/auth/login
Me: http://localhost:8000/api/v1/auth/me


## Тест: Получение истории

In [3]:
response = requests.get(HISTORY_ENDPOINT)
print(f"Статус код: {response.status_code}")

if response.status_code == 200:
    data = response.json()
    print(f"Успешно получена история")
    print(f"Всего записей: {data['total']}")
    print(f"Записей в ответе: {len(data['records'])}")
else:
    print(f"Ошибка: {response.json()}")

Статус код: 200
Успешно получена история
Всего записей: 131
Записей в ответе: 100


## Тест: Создание нескольких запросов для заполнения истории

In [4]:
# загрузим данные для тестовых запросов
df = pd.read_pickle("../datasets/clean_ptbxl_with_ecg_n_diagnostic_superclass.pkl")
print(f"Загружено записей: {len(df)}")

Загружено записей: 12766


In [5]:
def create_payload(row: pd.Series) -> dict:
    """Создает payload для запроса к API."""
    heart_axis = row.get("heart_axis", "NO_DATA")
    if pd.isna(heart_axis):
        heart_axis = "NO_DATA"
    
    return {
        "age": float(row["age"]),
        "sex": int(row["sex"]),
        "height": float(row["height"]) if pd.notna(row["height"]) else 170.0,
        "weight": float(row["weight"]) if pd.notna(row["weight"]) else 70.0,
        "heart_axis": heart_axis,
        "ecg_signal": row["ecg_signals"].tolist()
    }

In [6]:
# отправляем 5 успешных запросов
for i in range(5):
    payload = create_payload(df.iloc[i])
    response = requests.post(FORWARD_ENDPOINT, json=payload)
    if response.status_code == 200:
        result = response.json()
        print(f"Запрос {i+1}: {result['class_name']}")
    else:
        print(f"Запрос {i+1}: ошибка {response.status_code}")

print("Тестовые запросы отправлены")

Запрос 1: N
Запрос 2: N
Запрос 3: CS
Запрос 4: N
Запрос 5: N
Тестовые запросы отправлены


## Тест: Получение истории после запросов

In [8]:
response = requests.get(HISTORY_ENDPOINT)
print(f"Статус код: {response.status_code}")

if response.status_code == 200:
    data = response.json()
    print(f"\nУспешно получена история")
    print(f"Всего записей: {data['total']}")
    print(f"\nПоследние записи:")
    
    for record in data['records'][:5]:
        print(f"  ID: {record['id']} | "
              f"Время: {record['created_at']} | "
              f"Класс: {record['predicted_class_name']} | "
              f"Успех: {record['success']}")
else:
    print(f"Ошибка: {response.json()}")

Статус код: 200

Успешно получена история
Всего записей: 136

Последние записи:
  ID: 136 | Время: 2025-12-14T15:12:41.041143 | Класс: N | Успех: True
  ID: 135 | Время: 2025-12-14T15:12:40.913204 | Класс: N | Успех: True
  ID: 134 | Время: 2025-12-14T15:12:40.781496 | Класс: CS | Успех: True
  ID: 133 | Время: 2025-12-14T15:12:40.629181 | Класс: N | Успех: True
  ID: 132 | Время: 2025-12-14T15:12:40.306737 | Класс: N | Успех: True


## Тест: Пагинация

In [9]:
# получаем первые 2 записи
response = requests.get(HISTORY_ENDPOINT, params={"limit": 2, "offset": 0})
data1 = response.json()
print(f"\nПервая страница (limit=2, offset=0):")
print(f"  Всего записей: {data1['total']}")
print(f"  Получено записей: {len(data1['records'])}")
for r in data1['records']:
    print(f"    ID: {r['id']}")

# получаем следующие 2 записи
response = requests.get(HISTORY_ENDPOINT, params={"limit": 2, "offset": 2})
data2 = response.json()
print(f"\nВторая страница (limit=2, offset=2):")
print(f"  Всего записей: {data2['total']}")
print(f"  Получено записей: {len(data2['records'])}")
for r in data2['records']:
    print(f"    ID: {r['id']}")

print("\nПагинация работает корректно")


Первая страница (limit=2, offset=0):
  Всего записей: 136
  Получено записей: 2
    ID: 136
    ID: 135

Вторая страница (limit=2, offset=2):
  Всего записей: 136
  Получено записей: 2
    ID: 134
    ID: 133

Пагинация работает корректно


## Тест: Преобразование истории в DataFrame

In [10]:
response = requests.get(HISTORY_ENDPOINT, params={"limit": 100})
data = response.json()

history_df = pd.DataFrame(data['records'])
print(f"\nИстория преобразована в DataFrame")
print(f"Размер: {history_df.shape}")
print(f"\nКолонки: {history_df.columns.tolist()}")

history_df


История преобразована в DataFrame
Размер: (100, 13)

Колонки: ['id', 'created_at', 'age', 'sex', 'height', 'weight', 'heart_axis', 'predicted_class_index', 'predicted_class_name', 'predicted_class_description', 'success', 'error_message', 'request_type']


,id,created_at,age,sex,height,weight,heart_axis,predicted_class_index,predicted_class_name,predicted_class_description,success,error_message,request_type
0,136,2025-12-14T15:12:41.041143,43.0,1,170.0,44.0,NO_DATA,0,N,Normal ECG,True,None,single
1,135,2025-12-14T15:12:40.913204,22.0,1,170.0,56.0,NO_DATA,0,N,Normal ECG,True,None,single
2,134,2025-12-14T15:12:40.781496,54.0,0,170.0,83.0,LAD,5,CS,Conduction Disturbance + ST/T Change,True,None,single
3,133,2025-12-14T15:12:40.629181,37.0,1,170.0,69.0,NO_DATA,0,N,Normal ECG,True,None,single
4,132,2025-12-14T15:12:40.306737,19.0,0,170.0,70.0,NO_DATA,0,N,Normal ECG,True,None,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,41,2025-12-14T15:11:49.037116,56.0,0,170.0,70.0,LAD,5,CS,Conduction Disturbance + ST/T Change,True,None,batch
96,40,2025-12-14T15:11:48.975829,25.0,1,170.0,62.0,NO_DATA,0,N,Normal ECG,True,None,batch
97,39,2025-12-14T15:11:48.913348,20.0,0,170.0,81.0,NO_DATA,0,N,Normal ECG,True,None,batch
98,38,2025-12-14T15:11:48.851217,49.0,1,170.0,58.0,NO_DATA,0,N,Normal ECG,True,None,batch


## Тест: Анализ истории

In [11]:
if len(history_df) > 0:
    print(f"\nСтатистика:")
    print(f"Всего запросов: {len(history_df)}")
    print(f"Успешных: {history_df['success'].sum()}")
    print(f"С ошибками: {(~history_df['success']).sum()}")
    
    print(f"\nРаспределение по типам запросов:")
    print(history_df['request_type'].value_counts())
    
    print(f"\nРаспределение по предсказанным классам:")
    print(history_df['predicted_class_name'].value_counts())
else:
    print("История пуста")


Статистика:
Всего запросов: 100
Успешных: 100
С ошибками: 0

Распределение по типам запросов:
request_type
batch     85
single    15
Name: count, dtype: int64

Распределение по предсказанным классам:
predicted_class_name
N      68
MC      7
CS      4
S       4
M       4
H       4
C       3
MS      2
MHS     2
MHC     1
MH      1
Name: count, dtype: int64


## Тест: Проверка сохранения ошибочных запросов

In [12]:
# получаем текущее количество записей
response = requests.get(HISTORY_ENDPOINT)
before_count = response.json()['total']
print(f"Записей до ошибочного запроса: {before_count}")

# отправляем запрос с неверным heart_axis (должен вернуть 400)
bad_payload = {
    "age": 50,
    "sex": 1,
    "height": 175,
    "weight": 80,
    "heart_axis": "INVALID_VALUE",
    "ecg_signal": [[0.1] * 12 for _ in range(100)]
}

response = requests.post(FORWARD_ENDPOINT, json=bad_payload)
print(f"Статус ошибочного запроса: {response.status_code}")

# проверяем, что запрос НЕ записался в историю (т.к. валидация Pydantic происходит до эндпоинта)
response = requests.get(HISTORY_ENDPOINT)
after_count = response.json()['total']
print(f"Записей после ошибочного запроса: {after_count}")

# примечание: запросы с ошибками валидации Pydantic не попадают в эндпоинт,
# поэтому не сохраняются в историю

Записей до ошибочного запроса: 136
Статус ошибочного запроса: 400
Записей после ошибочного запроса: 136


## Тест: JWT Авторизация

In [13]:
# попытка входа с неверными данными
print("\nПопытка входа с неверными данными:")
response = requests.post(
    LOGIN_ENDPOINT,
    data={"username": "wrong", "password": "wrong"}
)
print(f"Статус код: {response.status_code}")
print(f"Ожидаемый: 401 (Unauthorized)")


Попытка входа с неверными данными:
Статус код: 401
Ожидаемый: 401 (Unauthorized)


In [14]:
# вход с правильными данными (admin/admin123)
response = requests.post(
    LOGIN_ENDPOINT,
    data={"username": "admin", "password": "admin123"}
)
print(f"Статус код: {response.status_code}")

if response.status_code == 200:
    token_data = response.json()
    ACCESS_TOKEN = token_data["access_token"]
    print(f"Токен получен: {ACCESS_TOKEN[:50]}...")
    print(f"Тип токена: {token_data['token_type']}")
else:
    print(f"Ошибка: {response.json()}")
    ACCESS_TOKEN = None

Статус код: 200
Токен получен: eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJzdWIiOiJhZ...
Тип токена: bearer


In [15]:
# проверка /auth/me с токеном
if ACCESS_TOKEN:
    response = requests.get(
        ME_ENDPOINT,
        headers={"Authorization": f"Bearer {ACCESS_TOKEN}"}
    )
    print(f"Статус код: {response.status_code}")
    if response.status_code == 200:
        user = response.json()
        print(f"Пользователь: {user['username']}, роль: {user['role']}")
    else:
        print(f"Ошибка: {response.json()}")

Статус код: 200
Пользователь: admin, роль: admin


## Тест: DELETE с правильным токеном

In [16]:
# попытка получить статистику без авторизации
print("\nGET /stats без токена:")
response = requests.get(STATS_ENDPOINT)
print(f"Статус код: {response.status_code}")
print(f"Ожидаемый: 401 (Unauthorized)")


GET /stats без токена:
Статус код: 401
Ожидаемый: 401 (Unauthorized)


## Тест: Доступ к защищённым эндпоинтам с токеном админа

In [17]:
# получаем токен
response = requests.post(
    LOGIN_ENDPOINT,
    data={"username": "admin", "password": "admin123"}
)
ACCESS_TOKEN = response.json()["access_token"]
headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

# получение статистики с авторизацией
print("\nGET /stats с токеном админа:")
response = requests.get(STATS_ENDPOINT, headers=headers)
print(f"Статус код: {response.status_code}")

if response.status_code == 200:
    stats = response.json()
    print(f"Статистика получена")
    print(f"Всего запросов: {stats['total_requests']}")
    if stats.get('processing_time'):
        pt = stats['processing_time']
        print(f"Среднее время обработки: {pt['mean_ms']:.2f} мс")
else:
    print(f"Ошибка: {response.json()}")


GET /stats с токеном админа:
Статус код: 200
Статистика получена
Всего запросов: 136
Среднее время обработки: 64.19 мс


In [18]:
# удаление истории с авторизацией
print("\nDELETE /history с токеном админа:")
response = requests.get(HISTORY_ENDPOINT)
before_count = response.json()['total']
print(f"Записей до удаления: {before_count}")

response = requests.delete(HISTORY_ENDPOINT, headers=headers)
print(f"Статус код: {response.status_code}")

if response.status_code == 200:
    result = response.json()
    print(f"{result['message']}")
    print(f"Удалено записей: {result['deleted_count']}")
else:
    print(f"Ошибка: {response.json()}")


DELETE /history с токеном админа:
Записей до удаления: 136
Статус код: 200
История успешно удалена
Удалено записей: 136


In [19]:
# проверяем что история пустая
response = requests.get(HISTORY_ENDPOINT)
after_count = response.json()['total']
print(f"Записей после удаления: {after_count}")

Записей после удаления: 0
